# <center>2048 In One Line Of Code</center>

This project started as an off handed boast to my sister that I could create the puzzle game 2048 in one line of code. Unnecicarily said, I had no idea it was possible at that point. However, I'm not one to give up on crazy ideas easily. 

Starting this project I assumed that creating the game in 1 line of code would impossible, it might be possible with 3: one line for a function declaration, another to return a recursive call to the same function, and a final line for the first function call. I had written a simular piece of code before to solve an interview question to find the nth fibinacci number. However this would be far more complex as the operations on the parameter would be more involved than subtraction.

In [17]:
def fib(n):
    return fib(n - 1) + fib(n - 2) if n > 1 else 1
fib(5)

8

with this structure in mind, lets take a look at what types of lines of code, cannot be in our program to indetify some constraints we're working with. First off, we cannot have any variable allocations, but latter on we will see how we can fake that. Secondly, we cannot have additonal functions declarations, destroying any hope of code redibilty. Thirdly, we can not use conventional loop, however originally i suspected that the for keyword could be used in list comprehentions. The final line of code doesn't use a single for loop, but could be rewritten to. And finally, we cannot not import libraries as commonly seen in most python files. By now, you'll have probably expected that I found a work around for that too.

Because we cannot define local variables, I found a proxy by misusing lambda functions. Instead of using the '=' symbol, we can instead call a lambda function with the values we want to assing as arguements. For example instead of the following

In [36]:
x = 1
y = 2
w, z = 3, 4
sum([x, y, w, z])

10

we could do

In [37]:
(lambda x: (lambda y: (lambda w, z: sum([x, y, w, z]))))(1)(2)(3, 4)

10

after realizing that you could allocate primatives using lambdas I realized that I should be able to name lambda functions using the same method. Using this methodology, we can rewrite our two line recursive fibonacci function to be the folling one liner

In [18]:
(lambda func, args: func(func, args))(lambda fib, n: fib(fib, n - 1) + fib(fib, n - 2) if n > 1 else 1, 5)

8

Unlike normal function where the function name is always in the scope of the body of the function, lambdas are unnamed, so a reference to the function has to be passed into itself. In other words if the lambda function we passed in was simply lambda n: fib(n - 1) + fib(n - 2) if n > 1 else 1 then fib would not be defined.

I first discovered the ability to import libraries without using the standard imports statements when trying to solve a homework question as follows: a professor has office hours for 2 hours. 2 students visit his office at random for half an hour exactly. What is the probability that they meet?

The problem can be solve by testing whether two uniformly distributed random variables are within .5. a simulation is provided bellow. Looking at the result I correctly guessed the answer to be 5/9, but because I had to show work of my homework(and one line python program arn't valid), here is how I solved it mathamtically for the currious.

In [26]:
import random
sum([1 if abs(random.uniform(0, 1.5) - random.uniform(0, 1.5)) < .5 else 0 for i in range(1000000)])/1000000

0.555232

Even as a novice programmer I was obsessed with code compactness over readibility, so I searched for a way to combine the import call into the computation line. While reading the documentation for built in python functions, I found out about __import__() which take in the name of a library as a string. Doing so the compressed simulation is the following line of code.

In [21]:
sum([1 if abs(__import__('random').uniform(0, 1.5) - __import__('random').uniform(0, 1.5)) < .5 else 0 for i in range(1000000)])/1000000

0.55489

With these 4 work arounds defined, I wish I could say I was able to just write the line of code the way code is normally written(from left to right), if we look our work around for variable assignment we can see how this is impossible.

In [35]:
(lambda x: (lambda y: (lambda w, z: sum([x, y, w, z]))))(1)(2)(3, 4)

10

The structure of the line of code will be much differenct than any conventional code structure. The line above, while much different then conventional code, still reads from left to right in the sense that y and its assigned value, 2, are to the left of the previous assignment x and its value 1. 
# rewrite
However, that has a major flaw in that ff instead we wanted to set y to be x + 1 instead of 2 the original line would fail as x is not defined. If we want to use previous variables, we have to instead using the following format

In [34]:
(lambda x: (lambda y: (lambda w, z: sum([x, y, w, z]))(3, 4))(x + 1))(1)

10

Doing so makes the line of code even more confusing as now the code is read outside inward versus natural languages which typicall use left to right and then top downward. 

In [38]:
def zero_locs(board):
	return list(filter(lambda x: board[x[0]][x[1]] == 0, list(zip(4*list(range(4)), list(sum(zip(*4*[list(range(4))]), ()))))))

In [39]:
def random_zero(board):
	return __import__('random').choice(zero_locs(board))

In [40]:
def refill_board(board):
	return (lambda x: list(map(lambda j: list(map(lambda i: __import__('random').choice([2, 4]) if (j, i) == (x[0], x[1]) else board[j][i], range(4))), range(4))))(random_zero(board))


In [41]:
def rotate_board(board, count):
	return rotate_board(list(map(list, zip(*board[::-1]))), count - 1) if count > 0 else board


In [ ]:
def shift_row(row):
	return list(filter(lambda i: i != 0, row)) + list(filter(lambda i: i == 0, row))


In [ ]:
def combine_row(row):
	return row if len(row) <= 1 else [row[0] + row[1]] + combine_row(row[2:]) + [0] if row[0] == row[1] else [row[0]] + combine_row(row[1:])


In [ ]:
def myprint(board):
	list(map(lambda row: print(row), board))

In [ ]:
def myprint2(boards):
	list(map(lambda board: myprint(row), boards))

In [ ]:
def twentyfourtyeight(board = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]):
	if not any(map(any, board)):
		board = refill_board(board)
	myprint(board)
	rotated_boards = list(map(lambda i: rotate_board(board, i), range(4)))
	combined_boards = list(map(lambda board: list(map(lambda row: combine_row(shift_row(row)), board)), rotated_boards))#good
	unrotated_boards = list(map(lambda i: rotate_board(combined_boards[i], 4 - i), range(4)))
	if unrotated_boards[1:] == unrotated_boards[:-1]:
		print('gameover')
		return
	input_str = input()
	input_ind = {'a':0, 's':1, 'd':2, 'w':3}[input_str]
	twentyfourtyeight(refill_board(refill_board(unrotated_boards[input_ind])) if sum(map(lambda x: x.count(0), unrotated_boards[input_ind])) > 1 and __import__('random').choice([True, False]) else refill_board(unrotated_boards[input_ind])) if unrotated_boards[input_ind] != board else twentyfourtyeight(board)

# How do you do one line class declarations?

In [84]:
(lambda X: X(1, 2) + X(1,2))(lambda x, y: type('X', (object,), {'__add__': lambda self, other: (self.x + other.x, self.y + other.y), 'x': x, 'y': y, 'foo': lambda self: 'foo'})())

(2, 4)

In [71]:
(lambda X: X())(type('X', (object,), {'foo': lambda self: 'foo'}))
#type of this is the line above

__main__.X

In [ ]:
(lambda make_X: type('X', (object,), diction))

# Can every Python file be transpiled to be one line?

In [80]:
type(1)(1)

1